In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import numpy as np
import calendar
import time
from matplotlib import colors
from sklearn import preprocessing
import cartopy.crs as crs
import cartopy.feature as cfeature
import scipy

In [ ]:
def read_data(init):
    years = ["2018", "2019", "2020", "2021"]
    savedir = "/home/aevans/ai2es/processed_data/frcst_err/"

    nam_fcast_and_error = []
    gfs_fcast_and_error = []
    hrrr_fcast_and_error = []

    for year in years:
        nam_fcast_and_error.append(
            pd.read_parquet(
                f"{savedir}nam_fcast_and_error_df_{init}z_{year}_mask_water_ny.parquet"
            )
        )
        gfs_fcast_and_error.append(
            pd.read_parquet(
                f"{savedir}gfs_fcast_and_error_df_{init}z_{year}_mask_water_ny.parquet"
            )
        )
        hrrr_fcast_and_error.append(
            pd.read_parquet(
                f"{savedir}hrrr_fcast_and_error_df_{init}z_{year}_mask_water_ny.parquet"
            )
        )

    nam_fcast_and_error_df = pd.concat(nam_fcast_and_error)
    gfs_fcast_and_error_df = pd.concat(gfs_fcast_and_error)
    hrrr_fcast_and_error_df = pd.concat(hrrr_fcast_and_error)

    # need to remove the random forecasts that have forecast hours 0
    # these are random because they only exist in the files that Ryan T. provided
    gfs_fcast_and_error_df = gfs_fcast_and_error_df[
        gfs_fcast_and_error_df["lead_time_ONLY_HOURS"] != 0.0
    ]
    nam_fcast_and_error_df = nam_fcast_and_error_df[
        nam_fcast_and_error_df["lead_time_ONLY_HOURS"] != 0.0
    ]
    hrrr_fcast_and_error_df = hrrr_fcast_and_error_df[
        hrrr_fcast_and_error_df["lead_time_ONLY_HOURS"] != 0.0
    ]
    return gfs_fcast_and_error_df, nam_fcast_and_error_df, hrrr_fcast_and_error_df

In [ ]:
def format_df(df):
    df = df[df['lead_time_HOUR'] <= 18]
    error_months = (
            df.groupby([df.time.dt.month, "station"])[
                f"t2m_error"
            ]
            .mean()
        ).to_frame().reset_index()
    return error_months

In [ ]:
def get_corrs(df, lulc, keys):
    df_pers = pd.DataFrame()
    df_rho = pd.DataFrame()
    df_tau = pd.DataFrame()
    df_p_score = pd.DataFrame()
    for i in np.arange(1,13):
        pers_ls = []
        rho_ls = []
        tau_ls = []
        p_score_ls = []
        df = months_df[months_df['time'] == i]
        for col, val in lulc.iteritems():
            # get correlations
            pers = scipy.stats.pearsonr(lulc[col], df['t2m_error'])[0]
            p_score = scipy.stats.pearsonr(lulc[col], df['t2m_error'])[1]
            rho = scipy.stats.spearmanr(lulc[col], df['t2m_error'])[0]
            tau = scipy.stats.kendalltau(lulc[col], df['t2m_error'])[0]

            # append
            pers_ls.append(pers)
            rho_ls.append(rho)
            tau_ls.append(tau)
            p_score_ls.append(p_score)

        df_pers1 = pd.DataFrame(index=keys)
        df_pers1[f'{i}'] = pers_ls
        df_rho1 = pd.DataFrame(index=keys)
        df_rho1[f'{i}'] = rho_ls
        df_tau1 = pd.DataFrame(index=keys)
        df_tau1[f'{i}'] = tau_ls
        df_p_score1 = pd.DataFrame(index=keys)
        df_p_score1[f'{i}'] = p_score_ls

        df_pers = pd.concat([df_pers, df_pers1], axis = 1)
        df_rho = pd.concat([df_rho, df_rho1], axis = 1)
        df_tau = pd.concat([df_tau, df_tau1], axis = 1)
        df_p_score = pd.concat([df_p_score, df_p_score1], axis=1)
        
    return df_pers, df_rho, df_tau, df_p_score


In [ ]:
def plot_heatmap(df, corr_type):
    fig, ax = plt.subplots(figsize=(12,9))
    ax.set_title(f'{corr_type} Correlation Coefficients by Aspect/Slope')
    ax = sns.heatmap(df.T, vmin = -1, vmax = 1, cmap=cm.seismic, annot = True)
    ax.set_ylabel('Month')
    ax.set_xlabel('Aspect/Slope')
    plt.tight_layout()

In [ ]:
init = "12"

gfs_fcast_and_error_df, nam_fcast_and_error_df, hrrr_fcast_and_error_df = read_data(
    init
)
gfs_fcast_and_error_df = gfs_fcast_and_error_df.reset_index()
nam_fcast_and_error_df = nam_fcast_and_error_df.reset_index()
hrrr_fcast_and_error_df = hrrr_fcast_and_error_df.reset_index()

In [ ]:
lulc = pd.read_csv('/home/aevans/nwp_bias/src/correlation/data/aspect_nam.csv')
lulc = lulc.drop(columns=['site', 'station'])
keys = lulc.keys()

In [ ]:
months_df = format_df(nam_fcast_and_error_df)

In [ ]:
months_df

# NAM

In [ ]:
df_pers, df_rho, df_tau, df_p_score = get_corrs(months_df, lulc, keys)

In [ ]:
plot_heatmap(df_pers, 'PERS')

In [ ]:
plot_heatmap(df_rho, 'RHO')

In [ ]:
plot_heatmap(df_tau, 'TAU')

In [ ]:
plot_heatmap(df_p_score, 'p_score')